# Spatial Interaction

The notebook for handling spatial interaction for the urban simulation module.

## Overview

The structure of this notebook is as follows.

- Preparation of required libraries and data and defining functions for spatial interaction models
- Predicting the parameters by estimating the total flows using the original data

## Preparation

First, we will prepare the required libraries.

In [12]:
# load libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
from math import sqrt

In [2]:
# load data

data_path = os.path.join('data', 'london_flows.csv')
od_tube_df = pd.read_csv(data_path)


In [3]:
od_tube_df

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167
...,...,...,...,...,...,...
61469,Woolwich Arsenal,Tower Gateway,127,7892,3342,13401.795549
61470,Woolwich Arsenal,West Ham,608,7892,5487,8701.454361
61471,Woolwich Arsenal,West India Quay,6,7892,400,9536.720451
61472,Woolwich Arsenal,West Silvertown,81,7892,893,5355.248554


In [23]:
# create matrix table of observed

od_matrix = od_tube_df.pivot_table(
    values = 'flows', index = 'station_origin', 
    columns = 'station_destination', aggfunc = 'sum', margins = True
)

od_matrix

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,599
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1224
Acton Town,NaN,NaN,NaN,3.0,17.0,NaN,35.0,0.0,NaN,11.0,...,77.0,3.0,6.0,9.0,NaN,0.0,NaN,0.0,NaN,3745
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,17.0,...,0.0,NaN,4.0,8.0,NaN,0.0,NaN,0.0,NaN,2886
Aldgate East,NaN,NaN,2.0,0.0,NaN,NaN,0.0,0.0,NaN,20.0,...,24.0,0.0,0.0,12.0,NaN,1.0,NaN,1.0,NaN,3172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,5.0,47.0,NaN,NaN,NaN,NaN,22.0,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4868
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530
Woodside Park,NaN,NaN,1.0,26.0,11.0,NaN,0.0,NaN,NaN,59.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3093


In [7]:
# define the spatial interaction models

def spatial_interaction(
        df: pd.DataFrame,
        subset = 'all',
        orig_field = 'station_origin',
        dest_field = 'station_destination',
        Oi_field = 'population',
        Dj_field = 'jobs',
        cij_field = 'distance',
        actual = 'flows',
        cost_function = 'pow'
):
    """
    Runs the models of spatial interaction and annotates results to the original dataframe.
    Returns the original dataframe with results annotated, along with the regression model for each of the spatial interaction model.
    Requires statsmodels.api as sm, statsmodels.formula.api as smf

    Parameters
    ----------
    df : pd.DataFrame
        pandas DataFrame that includes the data for OD analysis
    subset : list
        list of names of boroughs used for analysis, or 'all' (default) to use the whole dataset
    orig_field : str
        the name of column for origin
    dest_field : str
        the name of column for destination
    Oi_field : str
        the name of column for origin statistic
    Dj_field : str
        the name of column for destination statistic
    cij_field : str
        the name of column for distance statistic
    actual : str
        the name of column for the actual value observed
    cost_function : str
        string showing which cost function to use. 'exp' for negative exponential, 'pow' for inverse power

    Returns
    -------
    return_df
        a dataframe with annotated data
        adds the following 4 columns to the original dataframe:
        'unconstrained_est', 
        'origin_constrained_est', 
        'destination_constrained_est', 
        'doubly_constrained_est'

    models
        a dictionary of Generalized Linear Model wrapper objects, including 4 models indexed as:
        'unconstrained', 
        'origin_constrained', 
        'destination_constrained', 
        'doubly_constrained'
    }       

    """

    # create new dataframe with only the required rows and columns
    columns = [orig_field, dest_field, Oi_field, Dj_field, cij_field, actual]
    new_df = df[columns].copy().reset_index().drop(columns = 'index')
 
    # get rid of the internal flows for now
    new_df = new_df[new_df[orig_field] != new_df[dest_field]].copy()

    # subset if specified
    if (subset != 'all'):
        new_df = new_df[
            (new_df[orig_field].isin(subset)) & 
            (new_df[dest_field].isin(subset))
        ].copy()

    # get the log of origin and destination
    new_df['log_Oi'] = np.log(new_df[Oi_field])
    new_df['log_Dj'] = np.log(new_df[Dj_field])

    # get log of cost function
    # the inverse power cij ** (-beta), logged as -beta * np.log(cij)
    new_df['pow_cost'] = np.log(new_df[cij_field])

    # the negative exponential exp(-beta * cij), logged as -beta * cij
    new_df['exp_cost'] = new_df[cij_field]          

    # create formulas
    formulas = []

    for c in ['pow_cost', 'exp_cost']:
        formulas.extend(
            [
                f'{actual} ~ log_Oi + log_Dj + {c}',
                f'{actual} ~ {orig_field} + log_Dj + {c} -1',
                f'{actual} ~ log_Oi + {dest_field} + {c} -1',
                f'{actual} ~ {orig_field} + {dest_field} + {c} -1'
            ]
        ) 
    
    # run regression models
    models = []    
    
    for f in formulas:
        models.append(
            smf.glm(formula = f, data = new_df, family = sm.families.Poisson()).fit()
        )
        new_df[f'results']



    # ----- unconstrained model -----
        
    # create formula for unconstrained model
    formula_unconstrained = f'{actual} ~ log_Oi + log_Dj + log_cost'

    # run regression
    unco_sim = smf.glm(
        formula = formula_unconstrained,
        data = new_df,
        family = sm.families.Poisson()
    ).fit()

    # assign the parameter values
    K_unconstrained = unco_sim.params['Intercept']
    alpha_unconstrained = unco_sim.params['log_Oi']
    gamma_unconstrained = unco_sim.params['log_Dj']
    beta_unconstrained = -unco_sim.params['log_cost']

    # calculated the unconstrained value
    new_df['unconstrained_est'] = round(
        np.exp(
            (alpha_unconstrained * new_df['log_Oi'])
            + (gamma_unconstrained * new_df['log_Dj']) 
            - (beta_unconstrained * new_df['log_cost'])
            + K_unconstrained
        ), 0).astype(int)
    
    # append column to the returning dataframe
    columns.append('unconstrained_est')

    # ----- Origin Constrained Model -----

    # create formula for origin constrained model
    formula_origin_constrained = f'Total ~ {orig_field} + log_Dj + log_cost -1'

    orig_sim = smf.glm(
        formula = formula_origin_constrained,
        data = new_df,
        family = sm.families.Poisson()
    ).fit()

    # assign parameter values
    alpha_i_orco = pd.DataFrame(orig_sim.params).reset_index().rename(columns = {0:'alpha_i', 'index': 'coef'})
    gamma_orco = orig_sim.params['log_Dj']
    beta_orco = -orig_sim.params['log_cost']

    # fix indeces
    to_repl = ["(" + orig_field + ")\[", "(" + dest_field + ")\[", "\]"]
    for x in to_repl:
        alpha_i_orco['coef'] = alpha_i_orco['coef'].str.replace(x, '', regex = True)

    # join with original dataframe
    new_df = new_df.merge(alpha_i_orco, left_on = orig_field, right_on = 'coef', how = 'left').drop(columns = ['coef'])

    # calculated the origin-constrained estimated value
    new_df['origin_constrained_est'] = round(
        np.exp(
            new_df['alpha_i'] 
            + (gamma_orco * new_df['log_Dj']) 
            - (beta_orco * new_df['log_cost'])
        ), 0).astype(int)
    
    # append column to the returning dataframe
    columns.append('origin_constrained_est')

    # ----- Destination Constrained Model -----

    # create formula for destination constrained model
    formula_dest_constrained = f'Total ~ log_Oi + {dest_field} + log_cost -1'

    dest_sim = smf.glm(
        formula = formula_dest_constrained,
        data = new_df,
        family = sm.families.Poisson()
    ).fit()

    # assign parameter values
    alpha_deco = dest_sim.params['log_Oi']
    gamma_j_deco = pd.DataFrame(dest_sim.params).reset_index().rename(columns = {0:'gamma_j', 'index': 'coef'})
    beta_deco = -dest_sim.params['log_cost']

    # fix indeces
    for x in to_repl:
        gamma_j_deco['coef'] = gamma_j_deco['coef'].str.replace(x, '', regex = True)

    # join with original dataframe
    new_df = new_df.merge(gamma_j_deco, left_on = dest_field, right_on = 'coef', how = 'left').drop(columns = ['coef'])

    # calculated the origin-constrained estimated value
    new_df['destination_constrained_est'] = round(
        np.exp(
            (alpha_deco * new_df['log_Oi']) 
            + new_df['gamma_j'] 
            - (beta_deco * new_df['log_cost'])
        ), 0).astype(int)
    
    # append column to the returning dataframe
    columns.append('destination_constrained_est')

    # ----- Doubly Constrained Model -----

    # create formula for doubly constrained model
    formula_double_constrained = f'Total ~ {orig_field} + {dest_field} + log_cost -1'

    double_sim = smf.glm(
        formula = formula_double_constrained,
        data = new_df,
        family = sm.families.Poisson()
    ).fit()

    # assign parameter values
    coefs_dbl = pd.DataFrame(double_sim.params).reset_index().rename(columns = {0:'value', 'index': 'coef'})
    alpha_i_dbl = coefs_dbl[coefs_dbl.coef.str.startswith(orig_field)].rename(columns = {'value': 'alpha_i_dbl'})
    gamma_j_dbl = coefs_dbl[coefs_dbl.coef.str.startswith(dest_field)].rename(columns = {'value': 'gamma_j_dbl'})
    beta = -double_sim.params['log_cost']

    # calculated the origin-constrained estimated value
    new_df['doubly_constrained_est'] = np.round(double_sim.mu, 0)
    
    # append column to the returning dataframe
    columns.append('doubly_constrained_est')
    # create returning dataframe
    return_df = new_df[columns].copy()

    # create a dictionary of returning models
    models = {
        'unconstrained': unco_sim,
        'origin_constrained': orig_sim,
        'destination_constrained': dest_sim,
        'doubly_constrained': double_sim
    }

    # return dataframe and summaries for each model
    return return_df, models

In [8]:
# define the goodness-of-fit models

# R-squared
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2


In [39]:
od_df_new.sort_values('station_origin')

,station_origin,station_destination,flows,population,jobs,distance,pow_cost,exp_cost,pred_pow,pred_exp
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,9.003504,8131.525097,54.840694,76.846663
29,Abbey Road,Westferry,3,599,1250,5180.920874,8.552738,5180.920874,3.864746,5.631567
28,Abbey Road,West Silvertown,2,599,893,3880.048228,8.263603,3880.048228,4.444481,5.589901
27,Abbey Road,West Ham,130,599,5487,533.842422,6.280101,533.842422,55.521454,20.976473
26,Abbey Road,Stratford International,33,599,1546,2495.579492,7.822276,2495.579492,11.018008,13.569528
...,...,...,...,...,...,...,...,...,...,...
61449,Woolwich Arsenal,Gallions Reach,8,7892,557,12051.323946,9.396930,12051.323946,56.545841,49.837899
61450,Woolwich Arsenal,Heron Quays,21,7892,5975,10215.536655,9.231665,10215.536655,316.171508,389.905573
61451,Woolwich Arsenal,Island Gardens,0,7892,691,12483.746818,9.432183,12483.746818,42.181908,37.212171
61453,Woolwich Arsenal,Langdon Park,46,7892,1989,10534.263814,9.262388,10534.263814,129.402018,121.972824


In [46]:
od_tube_df[od_tube_df.station_origin == 'Abbey Road']

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167
5,Abbey Road,Crossharbour,1,599,1208,6686.475560
6,Abbey Road,Custom House,0,599,845,3824.855630
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909
8,Abbey Road,Cyprus,7,599,850,6532.099618
9,Abbey Road,Devons Road,1,599,611,3958.324171


In [31]:
od_df_new = od_tube_df[od_tube_df['station_origin'] != od_tube_df['station_destination']].copy()

# calculate cost functions

# inverse power cij ** (-beta), logged as -beta * np.log(cij)
od_df_new['pow_cost'] = np.log(od_df_new['distance'])
# the negative exponential exp(-beta * cij), logged as -beta * cij
od_df_new['exp_cost'] = od_df_new['distance']   

# do the calculation for doubly constrained models - to calculate the -beta and the optimal exponential relationship
formula_pow = 'flows ~ station_origin + station_destination + pow_cost'
formula_exp = 'flows ~ station_origin + station_destination + exp_cost'

dbl_pow_model = smf.glm(
    formula = formula_pow,
    data = od_df_new,
    family = sm.families.Poisson()
).fit()

od_df_new['pred_pow'] = dbl_pow_model.mu

dbl_exp_model = smf.glm(
    formula = formula_exp,
    data = od_df_new,
    family = sm.families.Poisson()
).fit()

od_df_new['pred_exp'] = dbl_exp_model.mu



In [51]:
print(f"beta for power model: {dbl_pow_model.params['pow_cost']}")
print(f"bta for exp model: {dbl_exp_model.params['exp_cost']}")

beta for power model: -0.9096317604932748
bta for exp model: -0.00015436969215638512


In [52]:
# show summary
dbl_pow_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  flows   No. Observations:                61456
Model:                            GLM   Df Residuals:                    60658
Model Family:                 Poisson   Df Model:                          797
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.7074e+05
Date:                Sun, 11 Feb 2024   Deviance:                   1.7693e+06
Time:                        02:59:28   Pearson chi2:                 2.47e+06
No. Iterations:                    27   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              7.7406      0.068    113.622      0.000       7.607       7.874
station_origin[T.Acton Central]                        1.6537      0.050     32.953      0.000       1.555       1.752
station_origin[T.Acton Town]                           1.4186      0.044     32.133      0.000       1.332       1.505
station_origin[T.Aldgate]                              0.1860      0.045      4.133      0.000       0.098       0.274
station_origin[T.Aldgate East]                         0.3275      0.045      7.333      0.000       0.240       0.415
station_origin[T.All Saints]                          -0.0749      0.055     -1.361      0.174      -0.183       0.033
station_origin[T.Alperton]                             1.0326      0.048     21.535      0.000       0.939       1.127
station_origin[T.Amersham]                             1.4704      0.050     29.234      0.000       1.372       1.569
station_origin[T.Anerley]                              1.7327      0.057     30.199      0.000       1.620       1.845
station_origin[T.Angel]                                0.6949      0.044     15.868      0.000       0.609       0.781
station_origin[T.Archway]                              1.5856      0.043     36.784      0.000       1.501       1.670
station_origin[T.Arnos Grove]                          1.4625      0.045     32.613      0.000       1.375       1.550
station_origin[T.Arsenal]                              0.4333      0.046      9.359      0.000       0.343       0.524
station_origin[T.Baker Street]                         1.2401      0.043     29.115      0.000       1.157       1.324
station_origin[T.Balham]                               2.3181      0.042     54.862      0.000       2.235       2.401
station_origin[T.Bank and Monument]                    2.3056      0.041     55.716      0.000       2.225       2.387
station_origin[T.Barbican]                            -0.7520      0.050    -15.080      0.000      -0.850      -0.654
station_origin[T.Barking]                              2.3345      0.042     54.965      0.000       2.251       2.418
station_origin[T.Barkingside]                          0.9643      0.050     19.474      0.000       0.867       1.061
station_origin[T.Barons Court]                         1.0649      0.044     24.253      0.000       0.979       1.151
station_origin[T.Battersea Park]                     -25.4120   1.98e+04     -0.001      0.999   -3.89e+04    3.89e+04
station_origin[T.Bayswater]                            0.0731      0.048      1.519      0.129      -0.021       0.167
station_origin[T.Beckton]                              1.4588      0.050     29.003      0.000       1.360  

In [53]:
dbl_exp_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  flows   No. Observations:                61456
Model:                            GLM   Df Residuals:                    60658
Model Family:                 Poisson   Df Model:                          797
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -8.5105e+05
Date:                Sun, 11 Feb 2024   Deviance:                   1.5299e+06
Time:                        02:59:53   Pearson chi2:                 2.02e+06
No. Iterations:                    27   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              1.3541      0.068     20.016      0.000       1.222       1.487
station_origin[T.Acton Central]                        1.6112      0.050     32.098      0.000       1.513       1.710
station_origin[T.Acton Town]                           1.4266      0.044     32.313      0.000       1.340       1.513
station_origin[T.Aldgate]                              0.1831      0.045      4.069      0.000       0.095       0.271
station_origin[T.Aldgate East]                         0.2714      0.045      6.079      0.000       0.184       0.359
station_origin[T.All Saints]                          -0.0706      0.055     -1.284      0.199      -0.178       0.037
station_origin[T.Alperton]                             1.3613      0.048     28.377      0.000       1.267       1.455
station_origin[T.Amersham]                             3.8703      0.052     74.342      0.000       3.768       3.972
station_origin[T.Anerley]                              1.7474      0.058     30.379      0.000       1.635       1.860
station_origin[T.Angel]                                0.5387      0.044     12.304      0.000       0.453       0.624
station_origin[T.Archway]                              1.4103      0.043     32.728      0.000       1.326       1.495
station_origin[T.Arnos Grove]                          1.7079      0.045     38.090      0.000       1.620       1.796
station_origin[T.Arsenal]                              0.2443      0.046      5.279      0.000       0.154       0.335
station_origin[T.Baker Street]                         1.1168      0.043     26.225      0.000       1.033       1.200
station_origin[T.Balham]                               2.2402      0.042     53.038      0.000       2.157       2.323
station_origin[T.Bank and Monument]                    2.3549      0.041     56.932      0.000       2.274       2.436
station_origin[T.Barbican]                            -0.7151      0.050    -14.343      0.000      -0.813      -0.617
station_origin[T.Barking]                              2.5705      0.042     60.549      0.000       2.487       2.654
station_origin[T.Barkingside]                          1.6763      0.050     33.840      0.000       1.579       1.773
station_origin[T.Barons Court]                         0.9504      0.044     21.652      0.000       0.864       1.036
station_origin[T.Battersea Park]                     -25.4746   2.06e+04     -0.001      0.999   -4.04e+04    4.04e+04
station_origin[T.Bayswater]                           -0.1227      0.048     -2.551      0.011      -0.217      -0.028
station_origin[T.Beckton]                              1.3128      0.050     26.138      0.000       1.214  

In [47]:
od_df_new.to_csv('data/london_flows_pred1.csv')

In [32]:
pred_exp_matrix = od_df_new.pivot_table(
    values = 'pred_exp', index = 'station_origin', 
    columns = 'station_destination', aggfunc = 'sum', margins = True
)

pred_exp_matrix

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.622014,5.990000e+02
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.469541,NaN,NaN,1.224000e+03
Acton Town,NaN,NaN,NaN,10.886281,9.791210,NaN,16.650312,0.070891,NaN,12.193996,...,40.470717,3.999578,2.137517,18.521361,NaN,0.216578,NaN,0.866650,NaN,3.745000e+03
Aldgate,NaN,NaN,1.436803,NaN,32.113611,NaN,NaN,0.006153,NaN,23.242135,...,6.912031,NaN,3.070407,2.092522,NaN,0.708506,NaN,0.966589,NaN,2.886000e+03
Aldgate East,NaN,NaN,1.511900,37.571466,NaN,NaN,0.369446,0.006473,NaN,24.450375,...,7.273300,0.718794,3.230022,2.201301,NaN,0.966044,NaN,1.016837,NaN,3.172000e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,1.538469,38.132867,44.441013,NaN,NaN,NaN,NaN,25.378139,...,7.401117,NaN,7.019473,NaN,NaN,NaN,NaN,NaN,NaN,4.868000e+03
Woodgrange Park,NaN,1.531378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.300000e+02
Woodside Park,NaN,NaN,2.019784,17.068105,15.347081,NaN,0.493552,NaN,NaN,25.327956,...,9.716580,NaN,4.439803,NaN,NaN,NaN,NaN,NaN,NaN,3.093000e+03


In [37]:
print(f"Exponential: {CalcRSquared(od_df_new['flows'], od_df_new['pred_exp'])}")
print(f"Power: {CalcRSquared(od_df_new['flows'], od_df_new['pred_pow'])}")


Exponential: 0.4979027747361467
Power: 0.4077121206134828
